# Cкрипт для парсинга диалогов из файла test_data.csv

# Главные задачи, которые должен выполнять скрипт:

- Извлекать реплики с приветствием – где менеджер поздоровался. 
- Извлекать реплики, где менеджер представил себя. 
- Извлекать имя менеджера. 
- Извлекать название компании. 
- Извлекать реплики, где менеджер попрощался.
- Проверять требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»


In [1]:
import pandas as pd
import nltk
import pymorphy2



In [2]:
pd.set_option('display.max_colwidth', None)

In [3]:
df = pd.read_csv('test_data.csv')
df.head()

,dlg_id,line_n,role,text
0,0,0,manager,Алло
1,0,1,client,Алло здравствуйте
2,0,2,manager,Добрый день
3,0,3,client,Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается
4,0,4,manager,Ага


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 480 entries, 0 to 479
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   dlg_id  480 non-null    int64 
 1   line_n  480 non-null    int64 
 2   role    480 non-null    object
 3   text    480 non-null    object
dtypes: int64(2), object(2)
memory usage: 15.1+ KB


Проверяем все уникальные значения в столбце role

In [5]:
df['role'].unique()

array(['manager', 'client'], dtype=object)

Из представленных данных можно сделать вывод, что в таблице перепутаны роли manager и client. Для дальнейшей работы это необходимо исправитть.

In [6]:
df.loc[df['role'] == 'client', 'role'] = 'manager_1'
df.loc[df['role'] == 'manager', 'role'] = 'client'
df.loc[df['role'] == 'manager_1', 'role'] = 'manager'
df.head()

,dlg_id,line_n,role,text
0,0,0,client,Алло
1,0,1,manager,Алло здравствуйте
2,0,2,client,Добрый день
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается
4,0,4,client,Ага


In [7]:
df['dlg_id'].unique()

array([0, 1, 2, 3, 4, 5])

Переводим данные в поле text в нижний регистр.

In [8]:
df['text'] = df['text'].str.lower()
df.head()

,dlg_id,line_n,role,text
0,0,0,client,алло
1,0,1,manager,алло здравствуйте
2,0,2,client,добрый день
3,0,3,manager,меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается
4,0,4,client,ага


In [9]:
#df[(df['role'] == 'manager') & (df['dlg_id'] == 2)][['text', 'dlg_id', 'role']]#.head(60)

Создаем датафрейм для записи результатов проверки и списки с ключевыми фразами.

In [10]:
result = pd.DataFrame(columns = None, index = df['dlg_id'].unique())
greeting = ['здравствуйте', 'добрый день']
goodbye = ['всего доброго', 'до свидания', 'всего хорошего', 'хорошего вечера']


Для проверки вхождения ключевых слов в реплики менеджеров и выборки фраз с этими словами в датафрейм пишем функцию check.

In [11]:
# Функция для проверки вхождения ключевых слов и выборки фраз в датафрейм
def check(dictionary, field):
    
    for dlg in df['dlg_id'].unique():
        for i in df[(df['role' ] == 'manager') & (df['dlg_id'] == dlg)]['text']:
            for n in dictionary:
                if n in i:
                    result.loc[dlg, field] = i
                    break
    
                    


                        
                        

Проводим проверку - поздоровался ли менеджер, если да, то заносим фразу с приветствием в датафрейм.

In [12]:
# Проверяем поздоровался ли менеджер
check(dictionary = greeting, field = 'greeting')
result

,greeting
0,алло здравствуйте
1,алло здравствуйте
2,алло здравствуйте
3,алло дмитрий добрый день
4,NaN
5,NaN


Для проверки того представился ли менеджер и извлечения его имени, сначала нужно проверить фразу на вхождение ключевых слов "меня", "зовут" и т.д., после чего к данной фразе можно применять библиотеку nltk для извлечения имени. Если применять без такой проверки, то будут выданы все имена упоминаемые менеджером в разговоре.

In [13]:
for dlg in df['dlg_id'].unique():
        for i in df[(df['role' ] == 'manager') & (df['dlg_id'] == dlg)]['text']:
            if ('меня' in i and 'зовут' in i) or ('моё' in i and 'имя' in i) or ('моe' in i and 'имя' in i):
                result.loc[dlg, 'introduc'] = i
                break 
        
            
result            

,greeting,introduc
0,алло здравствуйте,меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается
1,алло здравствуйте,меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается
2,алло здравствуйте,меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там
3,алло дмитрий добрый день,добрый меня максим зовут компания китобизнес удобно говорить
4,NaN,NaN
5,NaN,NaN


Анализируем фразы и извлекаем из них имена менеджеров.

In [14]:
prob_thresh = 0.4
morph = pymorphy2.MorphAnalyzer()
index = 0
for phrase in result['introduc']:
    if type(phrase) != float:
        for word in nltk.word_tokenize(phrase):
            for p in morph.parse(word):
                if 'Name' in p.tag and p.score >= prob_thresh:
                    result.loc[index, 'name'] = p.normal_form
    index += 1
                
result              



,greeting,introduc,name
0,алло здравствуйте,меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается,ангелина
1,алло здравствуйте,меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается,ангелина
2,алло здравствуйте,меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там,ангелина
3,алло дмитрий добрый день,добрый меня максим зовут компания китобизнес удобно говорить,максим
4,NaN,NaN,NaN
5,NaN,NaN,NaN


Логично предположить, что название фирмы должно прозвучать во фразе приветствия, в противном случае есть вероятность извлечь названия любых организаций, упоминающихся в разговоре. Для извлечения названия фирмы берем часть фразы, которая начинается от слова "компания" и до первого глагола или наречия.

In [15]:
prob_thresh = 0.4
morph = pymorphy2.MorphAnalyzer()

index = 0
for phrase in result['introduc']:
    if type(phrase) != float:
        if 'компания' in phrase:
            for word in phrase[phrase.index('компания'):].split(' '):
                for p in morph.parse(word):
                    if ('VERB' in p.tag and p.score >= prob_thresh) or ('ADVB' in p.tag and p.score >= prob_thresh):
                        #print(p.normal_form)
                        result.loc[index, 'firm'] = phrase[phrase.index('компания'):phrase.index(word)]
                        
                        break
                else:
                    continue
                break
    index += 1
    
result

,greeting,introduc,name,firm
0,алло здравствуйте,меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается,ангелина,компания диджитал бизнес
1,алло здравствуйте,меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается,ангелина,компания диджитал бизнес
2,алло здравствуйте,меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там,ангелина,компания диджитал бизнес
3,алло дмитрий добрый день,добрый меня максим зовут компания китобизнес удобно говорить,максим,компания китобизнес
4,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN


Проводим проверку - попрощался ли менеджер, если да, то заносим фразу в датафрейм.

In [16]:
check(dictionary = goodbye, field = 'goodbye')
result

,greeting,introduc,name,firm,goodbye
0,алло здравствуйте,меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается,ангелина,компания диджитал бизнес,всего хорошего до свидания
1,алло здравствуйте,меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается,ангелина,компания диджитал бизнес,до свидания
2,алло здравствуйте,меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там,ангелина,компания диджитал бизнес,NaN
3,алло дмитрий добрый день,добрый меня максим зовут компания китобизнес удобно говорить,максим,компания китобизнес,угу все хорошо да понедельника тогда всего доброго
4,NaN,NaN,NaN,NaN,во вторник все ну с вами да тогда до вторника до свидания
5,NaN,NaN,NaN,NaN,ну до свидания хорошего вечера


Проверяем требование к менеджеру: «В каждом диалоге обязательно необходимо поздороваться и попрощаться с клиентом»

In [17]:
result.loc[(result['goodbye'].isnull() == False) & (result['greeting'].isnull() == False), 'hello_goodbye'] = True
result.loc[(result['goodbye'].isnull() == True) | (result['greeting'].isnull() == True), 'hello_goodbye' ] = False

result

,greeting,introduc,name,firm,goodbye,hello_goodbye
0,алло здравствуйте,меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается,ангелина,компания диджитал бизнес,всего хорошего до свидания,True
1,алло здравствуйте,меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления а мы сели обратила внимание что у вас срок заканчивается,ангелина,компания диджитал бизнес,до свидания,True
2,алло здравствуйте,меня зовут ангелина компания диджитал бизнес звоню вам по поводу продления лицензии а мастера мы с вами сотрудничали по видео там,ангелина,компания диджитал бизнес,NaN,False
3,алло дмитрий добрый день,добрый меня максим зовут компания китобизнес удобно говорить,максим,компания китобизнес,угу все хорошо да понедельника тогда всего доброго,True
4,NaN,NaN,NaN,NaN,во вторник все ну с вами да тогда до вторника до свидания,False
5,NaN,NaN,NaN,NaN,ну до свидания хорошего вечера,False
